# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests



# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,23.94,83,100,0.45,US,1723250756
1,1,macau,22.2006,113.5461,32.06,89,20,4.12,MO,1723250757
2,2,malpica,43.3228,-8.8105,18.88,65,38,5.25,ES,1723250757
3,3,bilibino,68.0546,166.4372,16.39,38,100,5.36,RU,1723250757
4,4,avarua,-21.2078,-159.7750,23.03,56,20,3.09,CK,1723250543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True,
    color = "City",
    alpha = 0.6,
    size = "Humidity",
    tiles = "OSM")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria

#Get user input
desired_max_temp = float(input("What is your preferred Max Temp? "))
desired_humidity = float(input("What is your preferred Humidity? "))
desired_wind_speed = float(input("What is your preferred Wind Speed? "))

#Translate for code
desired_conditions = {
    'Max Temp': desired_max_temp,
    'Humidity': desired_humidity,
    'Wind Speed': desired_wind_speed
}

print(f"Your desired conditions: {desired_conditions}")

def calculate_similarity(row, desired_conditions):
    # Calculate the differences
    temp_diff = abs(row['Max Temp'] - desired_conditions['Max Temp'])
    humidity_diff = abs(row['Humidity'] - desired_conditions['Humidity'])
    wind_speed_diff = abs(row['Wind Speed'] - desired_conditions['Wind Speed'])
    
    total_diff = temp_diff + humidity_diff + wind_speed_diff
    
    return total_diff

city_data_df['similarity'] = city_data_df.apply(calculate_similarity, axis=1, desired_conditions=desired_conditions)

# Sort by similarity score (ascending, because a lower score means a closer match)
df_sorted = city_data_df.sort_values(by='similarity')

# Display the top matches
top_matches = df_sorted.head(10)
print(top_matches)

# Top matches on a map
plot = top_matches.hvplot.points(
    'Lng', 'Lat', geo=True, size=10, color='similarity', tiles='OSM', 
    hover_cols=['City', 'Max Temp', 'Humidity', 'Wind Speed', 'Country', 'Date']
)
hvplot.show(plot)

Your desired conditions: {'Max Temp': 20.0, 'Humidity': 0.0, 'Wind Speed': 6.0}
     City_ID         City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
138      138     red hill  33.9996  -79.2581     27.16         1          47   
547      547    mariental -24.6333   17.9667     12.70         9           0   
333      333       ghanzi -21.5667   21.7833     12.75        11           0   
468      468      malanje  -9.5402   16.3410     18.48        18           0   
453      453  tamanrasset  22.7850    5.5228     27.95        13           0   
331      331        opuwo -18.0607   13.8400     18.38        20           0   
448      448    kununurra -15.7667  128.7333     26.99        19         100   
378      378         afif  23.9065   42.9172     29.76        17          69   
385      385     san juan -31.5375  -68.5364     12.05        18          53   
459      459       newman  37.3138 -121.0208     36.99        11           0   

     Wind Speed Country        Date  si

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
print(hotel_df.head())

       City Country      Lat       Lng  Humidity Hotel Name
0    albany      US  42.6001  -73.9662        83           
1     macau      MO  22.2006  113.5461        89           
2   malpica      ES  43.3228   -8.8105        65           
3  bilibino      RU  68.0546  166.4372        38           
4    avarua      CK -21.2078 -159.7750        56           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df.head(10))

Starting hotel search
albany - nearest hotel: No hotel found
macau - nearest hotel: 富華粵海酒店 Fu Hua Guang Dong Hotel
malpica - nearest hotel: Pensión Panchito
bilibino - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
iqaluit - nearest hotel: Frobisher Inn
tapes - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
blackmans bay - nearest hotel: Villa Howden
sao joao da barra - nearest hotel: Pousada Mediterrâneo
yaren - nearest hotel: Aiwo Hotel
moyynkum - nearest hotel: No hotel found
magadan - nearest hotel: Гостиница «ВМ-Центр»
invercargill - nearest hotel: The Grand
adamstown - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
kafue - nearest hotel: Lwanginga guesthouse
fort bragg - nearest hotel: Airborne Inn Lodging
klyuchi - nearest hotel: No hotel found
village of oak creek (big park) - nearest hotel: The Views Inn
haiku-pauwela - nearest hotel: Inn At M

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
plot = hotel_df.hvplot.points(
    'Lng', 'Lat', geo=True, size=10, color='Humidity', tiles='OSM', 
    hover_cols=['City', 'Max Temp', 'Humidity', 'Wind Speed', 'Country', 'Date', 'Hotel Name']
)

# Display the map
hvplot.show(plot)

Launching server at http://localhost:52713
